In [1]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split


In [8]:
# Load the dataset from Problem 2
df = pd.read_parquet(r"C:\Users\Ego Men\Anaconda_codes\RiskThinking\Problem 1 - Raw Data Processing.parquet")

In [9]:
# Remove rows with NaN values
df.dropna(inplace=True)


In [10]:
# Calculate the features
df['vol_moving_avg'] = df.groupby('Symbol')['Volume'].rolling(30).mean().values
df['adj_close_rolling_med'] = df.groupby('Symbol')['Adj Close'].rolling(30).median().values


In [11]:
# Remove rows with NaN values
df.dropna(inplace=True)


In [12]:
# Select features and target
features = ['vol_moving_avg', 'adj_close_rolling_med']
target = 'Volume'

In [13]:
X = df[features]
y = df[target]

In [14]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
# Define the XGBoost model
params = {
    'n_estimators': 100,
    'max_depth': 6,
    'min_child_weight': 1,
    'gamma': 0,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'reg:squarederror',
    'nthread': 4,
    'seed': 42
}

In [16]:
# Train the model
model = xgb.XGBRegressor(**params)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, nthread=4, num_parallel_tree=None,
             predictor=None, ...)

In [17]:
# Make predictions on test data
y_pred = model.predict(X_test)

In [18]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

In [19]:
# Persist the trained model
model.save_model(r"C:\Users\Ego Men\Anaconda_codes\RiskThinking\Problem3_xgb.model")


In [20]:
# Persist the training metrics
with open(r"C:\Users\Ego Men\Anaconda_codes\RiskThinking\Problem3_xgb.model"  , 'w') as f:
    f.write(f'MAE: {mae}, MSE: {mse}')